In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
raw_votes = pd.read_csv('bids_votes.tsv', delimiter='\t', na_values='n/a')

In [3]:
# Votes were manually inspected for potential issues, and marked with the following key:
# K: Keep - possible multiple vote or attempted correction, but reasonable doubt exists
# E: Clear multiple vote
# D: Duplicate entry (e.g., two "First choice" responses)
# N: Voted not to ratify governance, not counted
# T: Submitted after deadline
#
# Votes that did not raise any flags had no marking and are loaded as np.nan
valid_votes = raw_votes[raw_votes.Decision.fillna('K') == 'K']
numeric_votes = valid_votes.replace({
    'First choice': 1,
    'Second choice': 2,
    'Third choice': 3,
    'Fourth choice': 4,
    'Fifth choice': 5,
    'Sixth choice': 6,
})
# Drop timestamp, decision and governance vote
votes = numeric_votes.T[3:].T
votes.columns = ['Pernet', 'Poldrack', 'Auer', 'Phillips', 'Pisner', 'Niso']

In [4]:
votes[:3]

,Pernet,Poldrack,Auer,Phillips,Pisner,Niso
0,3,1,2,4,6,5
1,3,1,6,4,5,2
2,5,4,1,NaN,6,2


In [5]:
# Validate that ordering strategy works with nans
for vote in votes.values.astype(np.float)[:3]:
    choices = votes.columns[~np.isnan(vote)]
    ranks = vote[~np.isnan(vote)]
    ballot = choices[np.argsort(ranks)]
    print(vote)
    print(ballot)

[3. 1. 2. 4. 6. 5.]
Index(['Poldrack', 'Auer', 'Pernet', 'Phillips', 'Niso', 'Pisner'], dtype='object')
[3. 1. 6. 4. 5. 2.]
Index(['Poldrack', 'Niso', 'Pernet', 'Phillips', 'Pisner', 'Auer'], dtype='object')
[ 5.  4.  1. nan  6.  2.]
Index(['Auer', 'Niso', 'Poldrack', 'Pernet', 'Pisner'], dtype='object')


In [6]:
ballots = []
for i, vote in enumerate(votes.values.astype(np.float), start=2):
    choices = votes.columns[~np.isnan(vote)]
    ranks = vote[~np.isnan(vote)]
    # Check if any duplicate entries missed manual inspection
    if len(ranks) != len(set(ranks)):
        print(i, vote)
    ballots.append(list(choices[np.argsort(ranks)]))

In [7]:
def count_votes(ballots):
    # Find first choices, dropping empty ballots (can happen after first prune)
    first_choice = [ballot[0] for ballot in ballots if ballot]
    # Sort first choices by number of votes, most votes first
    return sorted(
        Counter(first_choice).items(),
        key=lambda x: x[1],
        reverse=True)

def prune(ballots, remove_slate):
    # Filter the removed slate from all ballots
    return [[slate for slate in ballot if slate != remove_slate]
            for ballot in ballots]

In [8]:
count_votes(ballots)

[('Niso', 47),
 ('Poldrack', 42),
 ('Pernet', 42),
 ('Pisner', 15),
 ('Phillips', 9),
 ('Auer', 8)]

In [9]:
round1 = prune(ballots, 'Auer')
count_votes(round1)

[('Niso', 49),
 ('Poldrack', 46),
 ('Pernet', 42),
 ('Pisner', 15),
 ('Phillips', 10)]

In [10]:
round2 = prune(round1, 'Phillips')
count_votes(round2)

[('Niso', 53), ('Poldrack', 47), ('Pernet', 46), ('Pisner', 16)]

In [11]:
round3 = prune(round2, 'Pisner')
count_votes(round3)

[('Niso', 58), ('Poldrack', 55), ('Pernet', 49)]

In [12]:
round4 = prune(round3, 'Pernet')
count_votes(round4)

[('Niso', 90), ('Poldrack', 70)]